# Classification de documents d'opinions

Groupe E :
Nom - numéro d'étudiant

Darnala Baptiste -

Di Giovanni Thomas - 21505926

Duverger Eliott -

Pierre (dsl je connais pas ton nom de famille) -

# Pré-traitements

On commence par importer les données :

In [1]:
import pandas

movieComments = pandas.read_csv('Data/dataset.csv', sep='\t', header = None, encoding = "utf8")
movieCommentsLabels = pandas.read_csv('Data/labels.csv', sep='\t', header = None, encoding = "utf8")

Pré-traitements choisis :

1- Supression de caractères non ASCII

2- Suppression des contractions

3- Passage en minuscule

4- Supression de la ponctuation

5- Suppressions des stopwords

6- Lemmatisation

Pas de remplacement des nombres

In [4]:
import nltk
import unicodedata
import contractions
from IPython.display import display
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

def cleanText(commentString):   
    # Removing non ASCII characters
    commentString = unicodedata.normalize('NFKD', commentString).encode("ascii", "ignore").decode("utf-8", 'ignore')

    # Removing contractions
    commentString = contractions.fix(commentString, slang = True)

    # Tokenizing
    tokenizedText = word_tokenize(commentString)

    # Putting all words in lowercase
    tokenizedText = [word.lower() for word in tokenizedText]

    # Deleting ponctuations
    tokenizedText = [word for word in tokenizedText if word.isalpha()]

    # Removing stop words
    tokenizedText = [word for word in tokenizedText if not word in stopwords.words('english')]
    
    # Converting numbers
    #inflectEngine = inflect.engine()
    #newWords = []
    #for word in tokenizedText:
    #    if word.isdigit():
    #        newWords.append(inflectEngine.number_to_words(word))
    #    else:
    #        newWords.append(word)
    #tokenizedText = newWords

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    commentString = [lemmatizer.lemmatize(word, pos = 'v') for word in tokenizedText]

    # Turning back tokens into a string
    commentString = "".join([" " + i for i in tokenizedText]).strip()
    
    return commentString

ModuleNotFoundError: No module named 'contractions'

# Classifieurs

In [5]:
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Sickit learn met régulièrement à jour des versions et indique des futurs warnings
# Ces deux lignes permettent de ne pas les afficher
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

Définition des variables d'apprentissage et des variables à prédire

In [7]:
movieCommentsArray = movieComments.values
data = movieCommentsArray[:, 0] # X

movieCommentsLabelsArray = movieCommentsLabels.values
dataLabels = movieCommentsLabelsArray[:, 0] # Y

Vectorisation

In [8]:
for i in range(len(data)):
    data[i] = cleanText(data[i])

vectorizer = TfidfVectorizer(ngram_range = (1, 2), min_df = 0.05)
vectors = vectorizer.fit_transform(data)

data = vectors.toarray()

NameError: name 'contractions' is not defined

Découpage des données en jeu d'apprentissage (70%) et jeu de test (30%)

In [9]:
trainingSize = 0.7
testingSize = 0.3

trainingData, testingData, trainingDataLabels, testingDataLabels = train_test_split(data, dataLabels, train_size = trainingSize, test_size = testingSize)
# X_train,    X_test,      Y_train,            Y_test

Définition des classifieurs et leurs paramètres

In [10]:
classifiers = {
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'SVC': SVC()
}

parameters = {
    'SVC': [
        {'C': [0.001, 0.01, 0.1, 1, 10]},
        {'gamma': [0.001, 0.01, 0.1, 1]},
        {'kernel': ['linear', 'rbf']}
    ],
    
    'RandomForestClassifier': [
        
    ]
}

ValueError: could not convert string to float: "I keep watching this movie over and over and over. I have to watch it at least once a week. I am from Africa and looking at that movie taught me some things that I didn't even know about Africa. Denzel's movies are all full of lessons for people of walks of life. I wish he was my own brother. I have also seen and love your Masala Mississipi. What a thrilling situation. When Denzel was trying to hook his brother up on the job, reminds me of my teen ages when my brother was always mad with me about getting myself busy all the time. My brother was always caring for my old father and he wants to see me the same way too. By the way where did Denzel get that African accent from in the Cry Freedom movie? I have first seen that movie in Africa and I didn't know then that Denzel was American till I moved down here."

Grid search pour rechercher le meilleur classifieur avec les meilleurs paramètres

In [ ]:
class Model:
    def __init__(self, classifier, parameters, score):
        self.classifier = classifier
        self.parameters = parameters
        self.score = score

    def __repr__(self):
        return repr((self.classifier, self.parameters, self.score))

results = []
for key, value in classifiers.items():
    gridSearch = GridSearchCV(
        estimator = value,
        param_grid = parameters[key],
        scoring = "accuracy",
        cv = 5,
        n_jobs = -1,
        iid = True)

    gridSearch.fit(trainingData, trainingDataLabels)

    result = Model(key, gridSearch.best_score_, gridSearch.best_estimator_)
    results.append(result)

results = sorted(results, key = lambda result: result.score, reverse = True)

print("Results from best to worst: \n")
for result in results:
    print ("Classifier: ", result.parameters,
    " with score %0.2f " %result.score, '\n')

Classifieurs

In [ ]:
models = []
models.append(("SVC", SVC()))
models.append(("Random Forest", RandomForestClassifier()))

for name, model in models:
    kFold = KFold(n_splits = 10, random_state = 10)
    crossVal = cross_val_score(model, data, dataLabels, cv = kFold, scoring = "accuracy")
    print(name, ": ", crossVal.mean(), " (", crossVal.std(), ") \n")

Utilisation d'une pipeline pour sauvegarder le modèle

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([("vectorizer", TfidfVectorizer(preprocessor = cleanText, ngram_range = (1, 2))),
    ("classifier", classifiers[results[0].name])])

Sauvegarde du modèle dans un fichier pickle

In [ ]:
import pickle

pickle.dump(results[0], open("groupeE.pkl", 'wb'))